In [1]:
# import sys
# !{sys.executable} -m pip install ing_theme_matplotlib
# import gdal
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import qbstyles
from ing_theme_matplotlib import mpl_style
# import glacierml as gl
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

# Step 1: Organize the data

## A. Load and organize the Glathida dataset

In [8]:
# raw_dataset = pd.read_csv('/data/fast0/datasets/glathida-3.1.0/data/T.csv')
T = pd.read_csv('/home/sa42/data/glac/T_models/T.csv')
T.rename(columns = {
    'LAT':'T_LAT_CENT',
    'LON':'T_LON_CENT',
    'AREA':'T_AREA',
    'MEAN_SLOPE':'T_MEAN_SLOPE',
    'MEAN_THICKNESS':'T_MEAN_THICKNESS',
    'MAXIMUM_THICKNESS':'T_MAXIMUM_THICKNESS' 
},inplace=True)
TT = pd.read_csv('/home/sa42/data/glac/T_models/TT.csv')
TT.rename(columns = {
    'AREA':'TT_AREA',
    'MEAN_SLOPE':'TT_MEAN_SLOPE',
    'MEAN_THICKNESS':'TT_MEAN_THICKNESS',
    'MAXIMUM_THICKNESS':'TT_MAXIMUM_THICKNESS',   
},inplace=True)
TTT = pd.read_csv('/home/sa42/data/glac/T_models/TTT.csv')
TTT.rename(columns = {
    'ELEVATION':'POINT_ELEVATION',
    'THICKNESS':'POINT_THICKNESS',
},inplace=True)
df1 = pd.merge(T,TT, how = 'inner', on='GlaThiDa_ID')
glathida = pd.merge(df1,TTT, how = 'inner', on='GlaThiDa_ID')
glathida = glathida[[
#     'GlaThiDa_ID',
#     'GLACIER_DB',
#     'GLACIER_ID',
    'T_LAT_CENT',
    'T_LON_CENT',
    'T_AREA',
    'T_MEAN_SLOPE',
    'T_MEAN_THICKNESS',
    'T_MAXIMUM_THICKNESS',
    'LOWER_BOUND',
    'UPPER_BOUND',
    'TT_AREA',
    'TT_MEAN_SLOPE',
    'TT_MEAN_THICKNESS',
    'TT_MAXIMUM_THICKNESS',
    'POINT_LAT',
    'POINT_LON',
    'POINT_ELEVATION',
    'POINT_THICKNESS',   
]]

# filter data by searching for specific entries
# idx = glathida.index[glathida['GLACIER_DB'] == 'RGI']
# glathida = glathida.loc[idx]

# # ignore RGI version, e.g. RGI60
# glathida['GLACIER_ID'] = glathida['GLACIER_ID'].str[6:]
# glathida.rename(columns = {'GLACIER_ID':'RGIId'},inplace=True)
glathida.dropna()

,T_LAT_CENT,T_LON_CENT,T_AREA,T_MEAN_SLOPE,T_MEAN_THICKNESS,T_MAXIMUM_THICKNESS,LOWER_BOUND,UPPER_BOUND,TT_AREA,TT_MEAN_SLOPE,TT_MEAN_THICKNESS,TT_MAXIMUM_THICKNESS,POINT_LAT,POINT_LON,POINT_ELEVATION,POINT_THICKNESS
0,46.9595,9.3891,0.08068,22.0,15.0,36.0,2600,2650,0.01985,24.0,14,29.0,46.960842,9.390133,2624.0,15
1,46.9595,9.3891,0.08068,22.0,15.0,36.0,2600,2650,0.01985,24.0,14,29.0,46.960844,9.390126,2625.0,15
2,46.9595,9.3891,0.08068,22.0,15.0,36.0,2600,2650,0.01985,24.0,14,29.0,46.960847,9.390118,2625.0,15
3,46.9595,9.3891,0.08068,22.0,15.0,36.0,2600,2650,0.01985,24.0,14,29.0,46.960849,9.390111,2625.0,14
4,46.9595,9.3891,0.08068,22.0,15.0,36.0,2600,2650,0.01985,24.0,14,29.0,46.960851,9.390103,2624.0,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174523,39.2370,97.7550,2.96629,21.0,41.0,121.0,5100,5200,0.01600,32.0,0,0.0,39.235309,97.762576,4780.0,90
174524,39.2370,97.7550,2.96629,21.0,41.0,121.0,5100,5200,0.01600,32.0,0,0.0,39.235217,97.762203,4780.0,101
174525,39.2370,97.7550,2.96629,21.0,41.0,121.0,5100,5200,0.01600,32.0,0,0.0,39.235171,97.761845,4780.0,106
174526,39.2370,97.7550,2.96629,21.0,41.0,121.0,5100,5200,0.01600,32.0,0,0.0,39.235114,97.761459,4782.0,112


### separate out features - what will be trained to predict desired attribute

In [9]:
train_dataset = glathida.sample(frac=0.8, random_state=0)
test_dataset = glathida.drop(train_dataset.index)

train_features = train_dataset.copy()
test_features = test_dataset.copy()

#define label - attribute training to be picked
train_labels = train_features.pop("POINT_THICKNESS")
test_labels = test_features.pop("POINT_THICKNESS")

train_features.describe().transpose()[['mean', 'std']]

,mean,std
T_LAT_CENT,44.884618,8.184138
T_LON_CENT,29.534678,37.012584
T_AREA,1.071441,1.385139
T_MEAN_SLOPE,21.451132,5.426106
T_MEAN_THICKNESS,23.105245,16.478921
T_MAXIMUM_THICKNESS,75.322438,45.187764
LOWER_BOUND,2972.478636,686.613471
UPPER_BOUND,3017.794705,702.395229
TT_AREA,0.415120,0.625850
TT_MEAN_SLOPE,23.099206,9.076736


# Step 2: Setup the Model

### Establish normalization layers

In [10]:
normalizer = {}
variable_list = list(train_features)
for variable_name in variable_list:

    normalizer[variable_name] = preprocessing.Normalization(input_shape=[1,], axis=None)
    normalizer[variable_name].adapt(np.array(train_features[variable_name]))
    
    
normalizer['ALL'] = preprocessing.Normalization(axis=-1)
normalizer['ALL'].adapt(np.array(train_features))

### Single variable linear regression functions

In [11]:
# glacierml

def build_linear_model(normalizer):
    model = tf.keras.Sequential([
        normalizer,
        layers.Dense(1)
    ])

    model.compile(
        optimizer=tf.optimizers.Adam(learning_rate=0.1),
        loss='mean_absolute_error')
    
    return model

def plot_single_model_variable(x, y,feature_name):
    plt.scatter(train_features[feature_name], train_labels, label='Data')
    plt.plot(x, y, color='k', label='Predictions')
    plt.xlabel(feature_name)
    plt.ylabel('Avg Thickness (m)')
#     plt.xlim((0,20))
    plt.legend()
      
def plot_loss(history):
#     plt.subplots(figsize=(10,5))
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    #   plt.ylim([0, 10])
    plt.xlabel('Epoch')
    plt.ylabel('Error')
    plt.legend()
    plt.grid(True)

In [12]:
test_results = {}
linear_model = {}
linear_history = {}
linear_results = {}

# Linear regression

In [13]:
%%time
for variable_name in variable_list:

    linear_model[variable_name] = build_linear_model(normalizer[variable_name])
    linear_history[variable_name] = linear_model[variable_name].fit(
                                        train_features[variable_name], train_labels,        
                                        epochs=1000,
                                        verbose=0,
                                        validation_split = 0.2)
    
    
    linear_results[variable_name] = linear_model[variable_name].evaluate(
                                        test_features[variable_name],
                                        test_labels, verbose=0)

linear_results

In [ ]:
for variable_name in variable_list:    
    df = pd.DataFrame(linear_history[variable_name].history)
    dfs = df.loc[[df.last_valid_index()]]
    dfs.insert(0, 'Variable', [variable_name])
    
    print(dfs)
    print('')


In [ ]:
import matplotlib.pyplot as plt
from ing_theme_matplotlib import mpl_style
 
def plot(dark):
  mpl_style(dark)
  plt.plot([1, 3, 9, 5, 2, 1, 1], marker='o')
  plt.plot([4, 5, 5, 7, 9, 8, 6], marker='o')
 
  plt.show()
 
plot(dark=True)

In [ ]:
fig,ax=plt.subplots(2,6,figsize=(10,10))
for i, variable_name in enumerate(variable_list):
    mpl_style("dark")
    ax = plt.subplot(4,4,i+1)
    plt.subplot(4,4,i+1)
    plot_loss(linear_history[variable_name])
    ax.set_title(variable_name)
#     plt.savefig("/home/sa42/notebooks/glac/figs/GTP1_loss.eps")

# Linear regression with multiple inputs

In [ ]:
linear_model = build_linear_model(normalizer['ALL'])

history_full = linear_model.fit(
train_features, train_labels,        
   epochs=1000,
   verbose=0,
   validation_split = 0.2)

test_results['MULTI'] = linear_model.evaluate(
    test_features,
    test_labels, verbose=0)

test_results

In [ ]:
df = pd.DataFrame(history_full.history)
dfs = df.loc[[df.last_valid_index()]]
dfs.insert(0, 'Variable', 'Multi-Variable')
    
print(dfs)



In [ ]:
plot_loss(history_full)
# plt.savefig("/home/sa42/notebooks/glac/figs/GTP1_full_loss.eps")

# DNN regression

## Define regression functions

In [ ]:
def build_dnn_model(norm):
    model = keras.Sequential([
              norm,
              layers.Dense(64, activation='relu'),
              layers.Dense(64, activation='relu'),
              layers.Dense(1) ])

    model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.01))
    
    return model

def dnn_test_results(feature_name):
    dnn_test_results[feature_name] = dnn_model.evaluate(
        test_features[feature_name],
        test_labels, verbose=0)

In [ ]:
test_results = {}
dnn_model = {}
dnn_history = {}
dnn_results = {}

In [ ]:
for variable_name in variable_list:

    dnn_model[variable_name] = build_dnn_model(normalizer[variable_name])
    dnn_history[variable_name] = dnn_model[variable_name].fit(
                                        train_features[variable_name], train_labels,        
                                        epochs=1000,
                                        verbose=0,
                                        validation_split = 0.2)
    dnn_results[variable_name] = dnn_model[variable_name].evaluate(
                                        test_features[variable_name],
                                        test_labels, verbose=0)

dnn_results

In [ ]:
for variable_name in variable_list:    
    df = pd.DataFrame(dnn_history[variable_name].history)
    dfs = df.loc[[df.last_valid_index()]]
    dfs.insert(0, 'Variable', [variable_name])
    
    print(dfs)
    print('')


In [ ]:
fig,ax=plt.subplots(4,4,figsize=(10,10))
for i, variable_name in enumerate(variable_list):
    plt.subplot(4,4,i+1)
    xmax = np.max(train_features[variable_name])
    xmin = np.min(train_features[variable_name])
    x = tf.linspace(xmin, xmax, 101)
    y = dnn_model[variable_name].predict(x)
    plot_single_model_variable(x,y,variable_name)

In [ ]:
fig,ax=plt.subplots(4,4,figsize=(10,10))
for i, variable_name in enumerate(variable_list):
    ax = plt.subplot(4,4,i+1)
    plot_loss(dnn_history[variable_name])
    ax.set_title(variable_name)
#     plt.savefig("/home/sa42/notebooks/glac/figs/GTP1_dnn_loss.eps")

# Full model

In [ ]:
dnn_full_model = build_dnn_model(normalizer['ALL'])

In [ ]:
%%time
dnn_history_full = dnn_full_model.fit(
    train_features, train_labels,
    validation_split=0.2,
    verbose=0, epochs=1000)

In [ ]:
df = pd.DataFrame(dnn_history_full.history)
dfs = df.loc[[df.last_valid_index()]]
dfs.insert(0, 'Variable', 'Multi-Variable')
    
print(dfs)


In [ ]:
plot_loss(dnn_history_full)
# plt.savefig("/home/sa42/notebooks/glac/figs/GTP1_dnn_full_loss.png")

In [ ]:
y = dnn_full_model.predict(test_features)
plt.plot(test_labels,y,'.')
plt.plot((0,200),(0,200),'-')
plt.xlabel('True Thickness (m)')
plt.ylabel('Model Thickness (m)')
plt.xlim((0,200))
plt.ylim((0,200))
# plt.savefig("/home/sa42/notebooks/glac/figs/GTP1_res.EPS")